In [57]:
#Imports
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

#Importing the quantum models.
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2, TwoLocal
from qiskit_machine_learning.optimizers import COBYLA, SPSA, L_BFGS_B

#Importing the classical models.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#Loading sample datasets
from sklearn.datasets import load_iris, make_classification, load_breast_cancer, load_wine

#Imports for hyperparameter tuning.
#from itertools import product
#from sklearn.model_selection import ParameterGrid, cross_val_score
#from collections import defaultdict

#!pip install pandas

In [58]:
# Quantum imports
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2
from qiskit_machine_learning.optimizers import COBYLA, SPSA

# Classical imports for comparison
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Set random seeds for reproducibility
np.random.seed(42)

# ========================================
# FIXED DATA PREPARATION
# ========================================
print("=== CREATING CONSISTENT DATASET ===")

# Create the same synthetic dataset but with consistent preprocessing
X, y = make_classification(
    n_samples=150,
    n_features=4,
    n_redundant=0,
    n_informative=4,
    n_clusters_per_class=1,
    class_sep=0.8,  # Slightly increased separation
    random_state=42
)

# Consistent train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# CRITICAL: Apply consistent scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Dataset shape: {X.shape}")
print(f"Training samples: {len(X_train_scaled)}, Test samples: {len(X_test_scaled)}")
print(f"Class distribution - Train: {np.bincount(y_train)}, Test: {np.bincount(y_test)}")

=== CREATING CONSISTENT DATASET ===
Dataset shape: (150, 4)
Training samples: 120, Test samples: 30
Class distribution - Train: [60 60], Test: [15 15]


In [59]:
# ========================================
# CLASSICAL BASELINE (CONSISTENT)
# ========================================
print("\n=== CLASSICAL BASELINES ===")

classifiers = [
    ("Logistic Regression", LogisticRegression(random_state=42, max_iter=1000)),
    ("SVM", SVC(random_state=42, kernel='rbf')),
    ("Random Forest", RandomForestClassifier(n_estimators=100, random_state=42))
]

classical_results = {}
for name, clf in classifiers:
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    classical_results[name] = acc
    print(f"{name}: {acc:.4f}")


=== CLASSICAL BASELINES ===
Logistic Regression: 0.8667
SVM: 0.9667
Random Forest: 0.8000


# Dataset Selection & Classical Model Training
##### Currently, I imported already available datasets from scikit learn in order to make testing easier.

In [60]:
# ========================================
# DATASET OPTION 1: Synthetic Challenging Dataset
# ========================================
print("=== OPTION 1: Synthetic Challenging Dataset ===")
X_synth, y_synth = make_classification(
    n_samples=150,          # Small dataset
    n_features=4,           # 4 features (good for quantum)
    n_redundant=0,          # No redundant features
    n_informative=4,        # All features are informative
    n_clusters_per_class=1, # One cluster per class
    class_sep=0.6,          # Lower separation = harder problem
    random_state=42
)
# Quick classical baseline test
X_train, X_test, y_train, y_test = train_test_split(X_synth, y_synth, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)
synth_score = lr.score(X_test_scaled, y_test)
print(f"Logistic Regression accuracy on synthetic data: {synth_score:.3f}")

=== OPTION 1: Synthetic Challenging Dataset ===
Logistic Regression accuracy on synthetic data: 0.756


In [61]:
# ========================================
# DATASET OPTION 2: Wine Dataset (Reduced to Binary)
# ========================================
print("\n=== OPTION 2: Wine Dataset (Binary Classification) ===")
wine_data = load_wine()
X_wine, y_wine = wine_data.data, wine_data.target

# Make it binary: Class 0 vs Classes 1&2
y_wine_binary = (y_wine > 0).astype(int)

# Use only first 4 features to keep it manageable for quantum
X_wine_4feat = X_wine[:, :4]

X_train, X_test, y_train, y_test = train_test_split(X_wine_4feat, y_wine_binary, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)
wine_score = lr.score(X_test_scaled, y_test)
print(f"Logistic Regression accuracy on wine data: {wine_score:.3f}")


=== OPTION 2: Wine Dataset (Binary Classification) ===
Logistic Regression accuracy on wine data: 0.926


In [62]:
# ========================================
# DATASET OPTION 3: Breast Cancer (Subset of Features)
# ========================================
print("\n=== OPTION 3: Breast Cancer Dataset (4 features) ===")
cancer_data = load_breast_cancer()
X_cancer, y_cancer = cancer_data.data, cancer_data.target

# Select 4 features that are known to be less predictive
# (avoiding the most discriminative ones)
feature_indices = [2, 7, 12, 17]  # Some texture and fractal dimension features
X_cancer_4feat = X_cancer[:, feature_indices]

X_train, X_test, y_train, y_test = train_test_split(X_cancer_4feat, y_cancer, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)
cancer_score = lr.score(X_test_scaled, y_test)
print(f"Logistic Regression accuracy on cancer subset: {cancer_score:.3f}")


=== OPTION 3: Breast Cancer Dataset (4 features) ===
Logistic Regression accuracy on cancer subset: 0.918


##### Above, I have trained a classical logical regression model on pre-available datasets to check for accuracy and determine the model that returns an average accuracy. I specifically select a dataset which produces average to check if a quantum model can help improve that accuracy.

In [63]:
# ========================================
# RECOMMENDATION
# ========================================
print("\n=== RECOMMENDATION ===")
scores = [synth_score, wine_score, cancer_score]
names = ["Synthetic", "Wine", "Cancer subset"]

# Find the dataset with most "average" performance (closest to 0.75-0.85 range)
target_range = (0.75, 0.85)
best_idx = 0
best_distance = float('inf')

for i, score in enumerate(scores):
    if target_range[0] <= score <= target_range[1]:
        distance = abs(score - 0.8)  # Distance from ideal "average" score
        if distance < best_distance:
            best_distance = distance
            best_idx = i

print(f"Recommended dataset: {names[best_idx]} (accuracy: {scores[best_idx]:.3f})")
print("This provides a good baseline that quantum models might be able to improve upon.")


=== RECOMMENDATION ===
Recommended dataset: Synthetic (accuracy: 0.756)
This provides a good baseline that quantum models might be able to improve upon.


##### The synthetic dataset (Dataset no.1) has produced an average accuracy.

In [64]:
# ========================================
# READY-TO-USE DATASET VARIABLES
# ========================================
if best_idx == 0:
    X_final, y_final = X_synth, y_synth
    dataset_name = "Synthetic"
elif best_idx == 1:
    X_final, y_final = X_wine_4feat, y_wine_binary
    dataset_name = "Wine"
else:
    X_final, y_final = X_cancer_4feat, y_cancer
    dataset_name = "Cancer subset"

print(f"\nDataset '{dataset_name}' is ready to use:")
print(f"Features shape: {X_final.shape}")
print(f"Labels shape: {y_final.shape}")
print(f"Classes: {np.unique(y_final)}")


Dataset 'Synthetic' is ready to use:
Features shape: (150, 4)
Labels shape: (150,)
Classes: [0 1]


In [65]:
# Splitting the data.
X, y = X_final, y_final
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")
print(f"Feature dimensions: {X_train.shape[1]}") #should be limited, we will try with only 4 features for now.

Training samples: 120, Test samples: 30
Feature dimensions: 4


##### The number of qubits to be used in the simulation will be the same as the number of features being used in the dataset. For this exercise, I have chosen to use 4 features and 4 qubits to closely simulate realo-life situations of limited resources.

In [66]:
num_qubits = X_train.shape[1]  # use number of features as qubits
feature_map = ZZFeatureMap(num_qubits, reps=1)        # encodes classical data
ansatz = RealAmplitudes(num_qubits, reps=1)           # variational layers

In [67]:
# Creating the sampler.
sampler = StatevectorSampler()

# Create the SamplerQNN
qnn = SamplerQNN(
    circuit=feature_map.compose(ansatz),
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    sampler=sampler
)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


In [68]:
# Define VQC
optimizer = COBYLA(maxiter=50)

# Create VQC with the correct API
vqc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


##### Training a Variational Quantum Classifier, the quantum equivalent of a binary classifier.

In [69]:
#Training and evaluation

#VQC
print("Training VQC...")
vqc.fit(X_train, y_train)

print("Making predictions...")
y_pred = vqc.predict(X_test)

# Calculate accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Quantum Variational Classifier Accuracy: {acc:.4f}")

# Additional metrics
print(f"Predictions: {y_pred}")
print(f"Actual:      {y_test}")

Training VQC...
Making predictions...
Quantum Variational Classifier Accuracy: 0.5667
Predictions: [0 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 1 1 1 1 0 1 1]
Actual:      [1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 1]


##### The very first iteration produced an accuracy of 43%. Upon later iterations, the accuracy increased to 53% which is only as good as a randomised guess. Below, we attempt to improve this accuracy.

In [70]:
print("=== IMPROVING QUANTUM MODEL PERFORMANCE ===")

# -----------------------------
# Try 1: More iterations and deeper circuit
# -----------------------------
print("\n--- Attempt 1: Deeper circuit + More iterations ---")
num_features = X_train.shape[1]

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)  # More reps
ansatz = RealAmplitudes(num_qubits=num_features, reps=3)  # Deeper ansatz

sampler = StatevectorSampler()
optimizer = COBYLA(maxiter=200)  # More iterations

vqc1 = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

start_time = time.time()
vqc1.fit(X_train, y_train)
training_time1 = time.time() - start_time

y_pred1 = vqc1.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)
print(f"Accuracy: {acc1:.4f} (Training time: {training_time1:.1f}s)")

# -----------------------------
# Try 2: Different ansatz (EfficientSU2)
# -----------------------------
print("\n--- Attempt 2: EfficientSU2 ansatz ---")
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)
ansatz = EfficientSU2(num_qubits=num_features, reps=2)  # Different ansatz

optimizer = COBYLA(maxiter=150)

vqc2 = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

start_time = time.time()
vqc2.fit(X_train, y_train)
training_time2 = time.time() - start_time

y_pred2 = vqc2.predict(X_test)
acc2 = accuracy_score(y_test, y_pred2)
print(f"Accuracy: {acc2:.4f} (Training time: {training_time2:.1f}s)")

# -----------------------------
# Try 3: Different optimizer (SPSA)
# -----------------------------
print("\n--- Attempt 3: SPSA optimizer ---")
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)
ansatz = RealAmplitudes(num_qubits=num_features, reps=2)

optimizer = SPSA(maxiter=100, learning_rate=0.1, perturbation=0.1)

vqc3 = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

start_time = time.time()
vqc3.fit(X_train, y_train)
training_time3 = time.time() - start_time

y_pred3 = vqc3.predict(X_test)
acc3 = accuracy_score(y_test, y_pred3)
print(f"Accuracy: {acc3:.4f} (Training time: {training_time3:.1f}s)")

# -----------------------------
# Try 4: Multiple random starts (best practice)
# -----------------------------
print("\n--- Attempt 4: Multiple random initializations ---")
best_acc = 0
best_model = None
best_pred = None

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)
ansatz = RealAmplitudes(num_qubits=num_features, reps=2)
optimizer = COBYLA(maxiter=100)

for i in range(3):  # Try 3 different random starts
    print(f"  Random start {i+1}/3...", end=" ")

    vqc_temp = VQC(
        sampler=sampler,
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer
    )

    # Set different random seed for each attempt
    np.random.seed(42 + i * 10)

    vqc_temp.fit(X_train, y_train)
    y_pred_temp = vqc_temp.predict(X_test)
    acc_temp = accuracy_score(y_test, y_pred_temp)

    print(f"Accuracy: {acc_temp:.4f}")

    if acc_temp > best_acc:
        best_acc = acc_temp
        best_model = vqc_temp
        best_pred = y_pred_temp

print(f"Best accuracy from multiple starts: {best_acc:.4f}")

# -----------------------------
# Summary and Comparison
# -----------------------------
print("\n=== RESULTS SUMMARY ===")
results = [
    ("Original (shallow)", 0.4333),  # Your original result
    ("Deeper circuit", acc1),
    ("EfficientSU2", acc2),
    ("SPSA optimizer", acc3),
    ("Multiple starts", best_acc)
]

print("Method                | Accuracy")
print("-" * 35)
for method, acc in results:
    print(f"{method:<20} | {acc:.4f}")

# Find best performing model
best_method_idx = np.argmax([r[1] for r in results])
print(f"\nBest method: {results[best_method_idx][0]} with {results[best_method_idx][1]:.4f} accuracy")

# -----------------------------
# Classical Baseline for Comparison
# -----------------------------
print("\n=== CLASSICAL BASELINE COMPARISON ===")
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

classifiers = [
    ("Logistic Regression", LogisticRegression(random_state=42)),
    ("SVM", SVC(random_state=42)),
    ("Random Forest", RandomForestClassifier(n_estimators=50, random_state=42))
]

print("Classical Method      | Accuracy")
print("-" * 35)
for name, clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred_classical = clf.predict(X_test)
    acc_classical = accuracy_score(y_test, y_pred_classical)
    print(f"{name:<20} | {acc_classical:.4f}")

print(f"\nQuantum best:         | {results[best_method_idx][1]:.4f}")

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


=== IMPROVING QUANTUM MODEL PERFORMANCE ===

--- Attempt 1: Deeper circuit + More iterations ---


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Accuracy: 0.4333 (Training time: 171.2s)

--- Attempt 2: EfficientSU2 ansatz ---


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


...

Accuracy: 0.5667 (Training time: 219.5s)

--- Attempt 4: Multiple random initializations ---
  Random start 1/3... 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Accuracy: 0.4667
  Random start 2/3... 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Accuracy: 0.6000
  Random start 3/3... Accuracy: 0.5667
Best accuracy from multiple starts: 0.6000

=== RESULTS SUMMARY ===
Method                | Accuracy
-----------------------------------
Original (shallow)   | 0.4333
Deeper circuit       | 0.4333
EfficientSU2         | 0.4333
SPSA optimizer       | 0.5667
Multiple starts      | 0.6000

Best method: Multiple starts with 0.6000 accuracy

=== CLASSICAL BASELINE COMPARISON ===
Classical Method      | Accuracy
-----------------------------------
Logistic Regression  | 0.8000
SVM                  | 0.9000
Random Forest        | 0.8333

Quantum best:         | 0.6000


In [71]:
# -----------------------------
# Try 2: Different ansatz (EfficientSU2)
# -----------------------------
print("\n--- Attempt 2: EfficientSU2 ansatz ---")
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)
ansatz = EfficientSU2(num_qubits=num_features, reps=2)  # Different ansatz

optimizer = COBYLA(maxiter=150)

vqc2 = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

start_time = time.time()
vqc2.fit(X_train, y_train)
training_time2 = time.time() - start_time

y_pred2 = vqc2.predict(X_test)
acc2 = accuracy_score(y_test, y_pred2)
print(f"Accuracy: {acc2:.4f} (Training time: {training_time2:.1f}s)")

# -----------------------------
# Try 3: Different optimizer (SPSA)
# -----------------------------
print("\n--- Attempt 3: SPSA optimizer ---")
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)
ansatz = RealAmplitudes(num_qubits=num_features, reps=2)

optimizer = SPSA(maxiter=100, learning_rate=0.1, perturbation=0.1)

vqc3 = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

start_time = time.time()
vqc3.fit(X_train, y_train)
training_time3 = time.time() - start_time

y_pred3 = vqc3.predict(X_test)
acc3 = accuracy_score(y_test, y_pred3)
print(f"Accuracy: {acc3:.4f} (Training time: {training_time3:.1f}s)")

# -----------------------------
# Try 4: Multiple random starts (best practice)
# -----------------------------
print("\n--- Attempt 4: Multiple random initializations ---")
best_acc = 0
best_model = None
best_pred = None

feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)
ansatz = RealAmplitudes(num_qubits=num_features, reps=2)
optimizer = COBYLA(maxiter=100)

for i in range(3):  # Try 3 different random starts
    print(f"  Random start {i+1}/3...", end=" ")

    vqc_temp = VQC(
        sampler=sampler,
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer
    )

    # Set different random seed for each attempt
    np.random.seed(42 + i * 10)

    vqc_temp.fit(X_train, y_train)
    y_pred_temp = vqc_temp.predict(X_test)
    acc_temp = accuracy_score(y_test, y_pred_temp)

    print(f"Accuracy: {acc_temp:.4f}")

    if acc_temp > best_acc:
        best_acc = acc_temp
        best_model = vqc_temp
        best_pred = y_pred_temp

print(f"Best accuracy from multiple starts: {best_acc:.4f}")

# -----------------------------
# Summary and Comparison
# -----------------------------
print("\n=== RESULTS SUMMARY ===")
results = [
    ("Original (shallow)", 0.4333),  # Your original result
    ("Deeper circuit", acc1),
    ("EfficientSU2", acc2),
    ("SPSA optimizer", acc3),
    ("Multiple starts", best_acc)
]

print("Method                | Accuracy")
print("-" * 35)
for method, acc in results:
    print(f"{method:<20} | {acc:.4f}")

# Find best performing model
best_method_idx = np.argmax([r[1] for r in results])
print(f"\nBest method: {results[best_method_idx][0]} with {results[best_method_idx][1]:.4f} accuracy")

# -----------------------------
# Classical Baseline for Comparison
# -----------------------------
print("\n=== CLASSICAL BASELINE COMPARISON ===")

classifiers = [
    ("Logistic Regression", LogisticRegression(random_state=42)),
    ("SVM", SVC(random_state=42)),
    ("Random Forest", RandomForestClassifier(n_estimators=50, random_state=42))
]

print("Classical Method      | Accuracy")
print("-" * 35)
for name, clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred_classical = clf.predict(X_test)
    acc_classical = accuracy_score(y_test, y_pred_classical)
    print(f"{name:<20} | {acc_classical:.4f}")

print(f"\nQuantum best:         | {results[best_method_idx][1]:.4f}")

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.



--- Attempt 2: EfficientSU2 ansatz ---


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Accuracy: 0.5333 (Training time: 173.5s)

--- Attempt 3: SPSA optimizer ---


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Accuracy: 0.6333 (Training time: 217.0s)

--- Attempt 4: Multiple random initializations ---
  Random start 1/3... 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Accuracy: 0.6000
  Random start 2/3... 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Accuracy: 0.6333
  Random start 3/3... Accuracy: 0.5667
Best accuracy from multiple starts: 0.6333

=== RESULTS SUMMARY ===
Method                | Accuracy
-----------------------------------
Original (shallow)   | 0.4333
Deeper circuit       | 0.4333
EfficientSU2         | 0.5333
SPSA optimizer       | 0.6333
Multiple starts      | 0.6333

Best method: SPSA optimizer with 0.6333 accuracy

=== CLASSICAL BASELINE COMPARISON ===
Classical Method      | Accuracy
-----------------------------------
Logistic Regression  | 0.8000
SVM                  | 0.9000
Random Forest        | 0.8333

Quantum best:         | 0.6333


# **HYPERPARAMETER TUNING**
##### This part of the notebook is computing intensive. Please do not run this unless you are ready to wait a couple of hours.

In [72]:
import numpy as np
import time
from itertools import product
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid, cross_val_score
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2, TwoLocal
from qiskit_machine_learning.optimizers import COBYLA, SPSA, L_BFGS_B
import matplotlib.pyplot as plt
from collections import defaultdict


In [73]:
print("=== COMPREHENSIVE QUANTUM ML HYPERPARAMETER TUNING ===")

# Assume X_train, X_test, y_train, y_test are already defined

# -----------------------------
# 1. GRID SEARCH FOR CORE PARAMETERS
# -----------------------------
print("\n1. Grid Search for Core Quantum Parameters")
print("This will take a while - testing multiple combinations...")

# Define parameter grid
param_grid = {
    'feature_map_reps': [1, 2, 3],
    'ansatz_reps': [1, 2, 3, 4],
    'optimizer_maxiter': [50, 100, 200],
    'ansatz_type': ['RealAmplitudes', 'EfficientSU2']
}

print(f"Total combinations to test: {len(list(ParameterGrid(param_grid)))}")

=== COMPREHENSIVE QUANTUM ML HYPERPARAMETER TUNING ===

1. Grid Search for Core Quantum Parameters
This will take a while - testing multiple combinations...
Total combinations to test: 72


In [74]:
# Store results
grid_results = []
best_score = 0
best_params = None

# Grid search
for i, params in enumerate(ParameterGrid(param_grid)):
    if i % 10 == 0:
        print(f"  Progress: {i+1}/{len(list(ParameterGrid(param_grid)))}")

    # Create feature map
    feature_map = ZZFeatureMap(
        feature_dimension=4,
        reps=params['feature_map_reps']
    )

    # Create ansatz
    if params['ansatz_type'] == 'RealAmplitudes':
        ansatz = RealAmplitudes(num_qubits=4, reps=params['ansatz_reps'])
    else:
        ansatz = EfficientSU2(num_qubits=4, reps=params['ansatz_reps'])

    # Create optimizer
    optimizer = COBYLA(maxiter=params['optimizer_maxiter'])

    try:
        # Create and train VQC
        vqc = VQC(
            sampler=StatevectorSampler(),
            feature_map=feature_map,
            ansatz=ansatz,
            optimizer=optimizer
        )

        # Train with timeout protection
        start_time = time.time()
        vqc.fit(X_train, y_train)
        training_time = time.time() - start_time

        # Evaluate
        y_pred = vqc.predict(X_test)
        score = accuracy_score(y_test, y_pred)

        # Store results
        result = {
            'params': params.copy(),
            'score': score,
            'training_time': training_time
        }
        grid_results.append(result)

        # Update best
        if score > best_score:
            best_score = score
            best_params = params.copy()

    except Exception as e:
        print(f"    Error with params {params}: {str(e)}")
        continue

print(f"Grid search complete! Best score: {best_score:.4f}")
print(f"Best parameters: {best_params}")

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


  Progress: 1/72


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler

  Progress: 11/72


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler

...

  Progress: 61/72


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.
No gradient function provided, creating a gradient function. If your Sampler

  Progress: 71/72


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Grid search complete! Best score: 0.8000
Best parameters: {'ansatz_reps': 2, 'ansatz_type': 'EfficientSU2', 'feature_map_reps': 2, 'optimizer_maxiter': 50}


In [75]:
# -----------------------------
# 2. OPTIMIZER-SPECIFIC TUNING
# -----------------------------
print("\n2. Fine-tuning Optimizer Parameters")

optimizer_configs = [
    # COBYLA variants
    {
        'name': 'COBYLA_conservative',
        'optimizer': COBYLA(maxiter=200, tol=1e-8, disp=False)
    },
    {
        'name': 'COBYLA_aggressive',
        'optimizer': COBYLA(maxiter=100, tol=1e-4, disp=False)
    },
    # SPSA variants
    {
        'name': 'SPSA_slow_learning',
        'optimizer': SPSA(maxiter=150, learning_rate=0.01, perturbation=0.05)
    },
    {
        'name': 'SPSA_fast_learning',
        'optimizer': SPSA(maxiter=100, learning_rate=0.1, perturbation=0.2)
    },
    {
        'name': 'SPSA_adaptive',
        'optimizer': SPSA(maxiter=150, learning_rate=0.05, perturbation=0.1)
    },
]

optimizer_results = {}

# Use best architecture from grid search
if best_params:
    feature_map = ZZFeatureMap(feature_dimension=4, reps=best_params['feature_map_reps'])
    if best_params['ansatz_type'] == 'RealAmplitudes':
        ansatz = RealAmplitudes(num_qubits=4, reps=best_params['ansatz_reps'])
    else:
        ansatz = EfficientSU2(num_qubits=4, reps=best_params['ansatz_reps'])
else:
    # Fallback if grid search failed
    feature_map = ZZFeatureMap(feature_dimension=4, reps=2)
    ansatz = RealAmplitudes(num_qubits=4, reps=2)

for config in optimizer_configs:
    print(f"  Testing {config['name']}...", end=" ")

    try:
        vqc_opt = VQC(
            sampler=StatevectorSampler(),
            feature_map=feature_map,
            ansatz=ansatz,
            optimizer=config['optimizer']
        )

        start_time = time.time()
        vqc_opt.fit(X_train, y_train)
        training_time = time.time() - start_time

        y_pred = vqc_opt.predict(X_test)
        score = accuracy_score(y_test, y_pred)

        optimizer_results[config['name']] = {
            'score': score,
            'time': training_time
        }

        print(f"Score: {score:.4f}, Time: {training_time:.1f}s")

    except Exception as e:
        print(f"Failed: {str(e)}")

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.



2. Fine-tuning Optimizer Parameters
  Testing COBYLA_conservative... 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.6667, Time: 235.6s
  Testing COBYLA_aggressive... 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.3000, Time: 117.5s
  Testing SPSA_slow_learning... 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.4667, Time: 354.0s
  Testing SPSA_fast_learning... 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.5000, Time: 236.4s
  Testing SPSA_adaptive... Score: 0.5667, Time: 354.9s


In [76]:
# -----------------------------
# 3. RANDOM SEARCH WITH BUDGET
# -----------------------------
print("\n3. Random Search for Fine-tuning")

def random_search(n_iterations=20):
    """Random search with larger parameter space"""

    random_results = []

    for i in range(n_iterations):
        # Random parameter selection
        params = {
            'feature_map_reps': np.random.choice([1, 2, 3, 4]),
            'ansatz_reps': np.random.choice([1, 2, 3, 4, 5]),
            'optimizer': np.random.choice(['COBYLA', 'SPSA']),
            'maxiter': np.random.choice([50, 75, 100, 150, 200]),
        }

        # COBYLA-specific parameters
        if params['optimizer'] == 'COBYLA':
            params['tol'] = 10**np.random.uniform(-8, -3)
            optimizer = COBYLA(maxiter=params['maxiter'], tol=params['tol'])

        # SPSA-specific parameters
        else:
            params['learning_rate'] = 10**np.random.uniform(-2, -0.5)
            params['perturbation'] = 10**np.random.uniform(-2, -0.5)
            optimizer = SPSA(
                maxiter=params['maxiter'],
                learning_rate=params['learning_rate'],
                perturbation=params['perturbation']
            )

        print(f"  Random trial {i+1}/{n_iterations}: ", end="")

        try:
            # Create components
            feature_map = ZZFeatureMap(feature_dimension=4, reps=params['feature_map_reps'])
            ansatz_type = np.random.choice(['RealAmplitudes', 'EfficientSU2'])

            if ansatz_type == 'RealAmplitudes':
                ansatz = RealAmplitudes(num_qubits=4, reps=params['ansatz_reps'])
            else:
                ansatz = EfficientSU2(num_qubits=4, reps=params['ansatz_reps'])

            params['ansatz_type'] = ansatz_type

            # Train and evaluate
            vqc_random = VQC(
                sampler=StatevectorSampler(),
                feature_map=feature_map,
                ansatz=ansatz,
                optimizer=optimizer
            )

            vqc_random.fit(X_train, y_train)
            y_pred = vqc_random.predict(X_test)
            score = accuracy_score(y_test, y_pred)

            random_results.append({
                'params': params,
                'score': score
            })

            print(f"Score: {score:.4f}")

        except Exception as e:
            print(f"Failed: {str(e)[:50]}...")

    return random_results

random_results = random_search(n_iterations=15)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.



3. Random Search for Fine-tuning
  Random trial 1/15: 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.6000
  Random trial 2/15: 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


...

Score: 0.4000
  Random trial 13/15: 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.4667
  Random trial 14/15: 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.5667
  Random trial 15/15: Score: 0.5667


In [77]:
# -----------------------------
# 4. ADAPTIVE PARAMETER TUNING
# -----------------------------
print("\n4. Adaptive Parameter Tuning (Bayesian-style)")

def adaptive_tuning(n_iterations=10):
    """Simple adaptive tuning based on previous results"""

    # Start with best known parameters
    if best_params:
        current_best = best_params.copy()
        current_score = best_score
    else:
        current_best = {'feature_map_reps': 2, 'ansatz_reps': 2, 'ansatz_type': 'RealAmplitudes'}
        current_score = 0.4

    adaptive_results = []

    for i in range(n_iterations):
        print(f"  Adaptive iteration {i+1}/{n_iterations}: ", end="")

        # Generate candidate by perturbing current best
        candidate = current_best.copy()

        # Randomly perturb one parameter
        param_to_change = np.random.choice(['feature_map_reps', 'ansatz_reps', 'ansatz_type'])

        if param_to_change == 'feature_map_reps':
            candidate['feature_map_reps'] = max(1, min(4, current_best['feature_map_reps'] + np.random.choice([-1, 0, 1])))
        elif param_to_change == 'ansatz_reps':
            candidate['ansatz_reps'] = max(1, min(5, current_best['ansatz_reps'] + np.random.choice([-1, 0, 1])))
        else:
            candidate['ansatz_type'] = np.random.choice(['RealAmplitudes', 'EfficientSU2'])

        try:
            # Test candidate
            feature_map = ZZFeatureMap(feature_dimension=4, reps=candidate['feature_map_reps'])
            if candidate['ansatz_type'] == 'RealAmplitudes':
                ansatz = RealAmplitudes(num_qubits=4, reps=candidate['ansatz_reps'])
            else:
                ansatz = EfficientSU2(num_qubits=4, reps=candidate['ansatz_reps'])

            vqc_adaptive = VQC(
                sampler=StatevectorSampler(),
                feature_map=feature_map,
                ansatz=ansatz,
                optimizer=COBYLA(maxiter=120)
            )

            vqc_adaptive.fit(X_train, y_train)
            y_pred = vqc_adaptive.predict(X_test)
            score = accuracy_score(y_test, y_pred)

            adaptive_results.append({
                'iteration': i,
                'params': candidate,
                'score': score
            })

            # Update current best if improved
            if score > current_score:
                current_best = candidate.copy()
                current_score = score
                print(f"NEW BEST! Score: {score:.4f}")
            else:
                print(f"Score: {score:.4f}")

        except Exception as e:
            print(f"Failed: {str(e)[:30]}...")

    return adaptive_results, current_best, current_score

adaptive_results, final_best_params, final_best_score = adaptive_tuning()

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.



4. Adaptive Parameter Tuning (Bayesian-style)
  Adaptive iteration 1/10: 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.3333
  Adaptive iteration 2/10: 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


...

Score: 0.4667
  Adaptive iteration 8/10: 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.6000
  Adaptive iteration 9/10: 

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Score: 0.4000
  Adaptive iteration 10/10: Score: 0.4333


In [78]:
# -----------------------------
# RESULTS SUMMARY
# -----------------------------
print("\n" + "="*60)
print("HYPERPARAMETER TUNING RESULTS SUMMARY")
print("="*60)

# Collect all scores
all_scores = []

# Grid search results
if grid_results:
    grid_best = max(grid_results, key=lambda x: x['score'])
    all_scores.append(('Grid Search', grid_best['score']))

# Optimizer tuning results
if optimizer_results:
    opt_best = max(optimizer_results.items(), key=lambda x: x[1]['score'])
    all_scores.append(('Optimizer Tuning', opt_best[1]['score']))

# Random search results
if random_results:
    random_best = max(random_results, key=lambda x: x['score'])
    all_scores.append(('Random Search', random_best['score']))

# Adaptive results
all_scores.append(('Adaptive Tuning', final_best_score))

# Display results
print("\nMethod                | Best Score | Improvement")
print("-" * 50)
baseline = 0.4333
for method, score in all_scores:
    improvement = score - baseline
    print(f"{method:<18} | {score:.4f}    | +{improvement:.4f}")

# Overall best
if all_scores:
    overall_best = max(all_scores, key=lambda x: x[1])
    total_improvement = overall_best[1] - baseline
    print(f"\nBEST OVERALL: {overall_best[0]} with {overall_best[1]:.4f} accuracy")
    print(f"Total improvement: +{total_improvement:.4f} ({(total_improvement/baseline)*100:.1f}%)")

    print(f"\nFinal best parameters: {final_best_params}")


HYPERPARAMETER TUNING RESULTS SUMMARY

Method                | Best Score | Improvement
--------------------------------------------------
Grid Search        | 0.8000    | +0.3667
Optimizer Tuning   | 0.6667    | +0.2334
Random Search      | 0.6000    | +0.1667
Adaptive Tuning    | 0.8000    | +0.3667

BEST OVERALL: Grid Search with 0.8000 accuracy
Total improvement: +0.3667 (84.6%)

Final best parameters: {'ansatz_reps': 2, 'ansatz_type': 'EfficientSU2', 'feature_map_reps': 2, 'optimizer_maxiter': 50}


In [79]:
import os
import pickle
import joblib
from datetime import datetime
import json

# ===========================================
# Creating a dedicated folder for models
# ===========================================

# Get current working directory (where your notebook is running)
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Create models folder
models_folder = "extracted_models"
models_path = os.path.join(current_dir, models_folder)
os.makedirs(models_path, exist_ok=True)
print(f"Models will be saved in: {models_path}")

Current working directory: C:\Users\safur\Projects\quantum-notebook
Models will be saved in: C:\Users\safur\Projects\quantum-notebook\extracted_models


In [81]:
# Use the same dataset and split parameters from your successful quantum training
X, y = X_final, y_final  # These should still be available from earlier cells

# Recreate the exact same train-test split used throughout your notebook
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Apply the same scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verify (this should show 120 and 30 samples)
print(f"Fixed - Training: {X_train_scaled.shape}, Test: {X_test_scaled.shape}")
print(f"Labels - Training: {y_train.shape}, Test: {y_test.shape}")
print("Data is now consistent for classical model training")

Fixed - Training: (120, 4), Test: (30, 4)
Labels - Training: (120,), Test: (30,)
Data is now consistent for classical model training


In [82]:
# ===========================================
# Extracting Classical Models
# ===========================================

print("\nExtracting Classical Models...")

# Train fresh classical models on your current data (to ensure they exist)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Using your existing X_train_scaled, X_test_scaled, y_train, y_test
classical_models = {}

# Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_accuracy = accuracy_score(y_test, lr_pred)
classical_models['logistic_regression'] = {
    'model': lr_model,
    'accuracy': lr_accuracy,
    'predictions': lr_pred
}

# SVM
svm_model = SVC(random_state=42, kernel='rbf')
svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_pred)
classical_models['svm'] = {
    'model': svm_model,
    'accuracy': svm_accuracy,
    'predictions': svm_pred
}

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_pred = rf_model.predict(X_test_scaled)
rf_accuracy = accuracy_score(y_test, rf_pred)
classical_models['random_forest'] = {
    'model': rf_model,
    'accuracy': rf_accuracy,
    'predictions': rf_pred
}

print("Classical models trained and ready for extraction")


Extracting Classical Models...
Classical models trained and ready for extraction


In [83]:
# ===========================================
# STEP 3: Save Classical Models
# ===========================================

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
classical_files = []

for model_name, model_data in classical_models.items():
    try:
        # Create complete model package
        model_package = {
            'model': model_data['model'],
            'accuracy': model_data['accuracy'],
            'model_type': 'classical',
            'model_name': model_name,
            'timestamp': timestamp,
            'training_samples': len(X_train_scaled),
            'test_samples': len(X_test_scaled),
            'features': X_train_scaled.shape[1]
        }
        
        # Save with joblib (better for sklearn models)
        filename = f"{model_name}_{timestamp}.joblib"
        filepath = os.path.join(models_path, filename)
        
        joblib.dump(model_package, filepath)
        classical_files.append(filepath)
        
        print(f"Saved: {model_name} -> {filename} (Accuracy: {model_data['accuracy']:.4f})")
        
    except Exception as e:
        print(f"Failed to save {model_name}: {str(e)}")

# ===========================================
# STEP 4: Find and Extract Best Quantum Model
# ===========================================

print("\n Extracting Best Quantum Model...")

# Find the best quantum model from your hyperparameter tuning results
best_quantum_model = None
best_quantum_accuracy = 0
best_quantum_params = None

# Check if you have results from hyperparameter tuning
if 'grid_results' in globals() and grid_results:
    # Find best from grid search
    best_result = max(grid_results, key=lambda x: x['score'])
    best_quantum_accuracy = best_result['score']
    best_quantum_params = best_result['params']
    print(f"Best quantum model found: {best_quantum_accuracy:.4f} accuracy")
    
    # Recreate the best model
    from qiskit.primitives import StatevectorSampler
    from qiskit_machine_learning.algorithms.classifiers import VQC
    from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes, EfficientSU2
    from qiskit_machine_learning.optimizers import COBYLA
    
    try:
        # Recreate feature map
        feature_map = ZZFeatureMap(
            feature_dimension=4,
            reps=best_quantum_params['feature_map_reps']
        )
        
        # Recreate ansatz
        if best_quantum_params['ansatz_type'] == 'RealAmplitudes':
            ansatz = RealAmplitudes(num_qubits=4, reps=best_quantum_params['ansatz_reps'])
        else:
            ansatz = EfficientSU2(num_qubits=4, reps=best_quantum_params['ansatz_reps'])
        
        # Recreate optimizer
        optimizer = COBYLA(maxiter=best_quantum_params['optimizer_maxiter'])
        
        # Create and train the best model
        best_quantum_model = VQC(
            sampler=StatevectorSampler(),
            feature_map=feature_map,
            ansatz=ansatz,
            optimizer=optimizer
        )
        
        print("Retraining best quantum model...")
        best_quantum_model.fit(X_train_scaled, y_train)
        
        # Verify accuracy
        quantum_pred = best_quantum_model.predict(X_test_scaled)
        verified_accuracy = accuracy_score(y_test, quantum_pred)
        print(f"Verified quantum model accuracy: {verified_accuracy:.4f}")
        
    except Exception as e:
        print(f"Failed to recreate best quantum model: {str(e)}")
        # Fall back to any available quantum model
        if 'vqc3' in globals():
            best_quantum_model = vqc3
            quantum_pred = best_quantum_model.predict(X_test_scaled)
            best_quantum_accuracy = accuracy_score(y_test, quantum_pred)
            best_quantum_params = {'fallback': 'using_vqc3'}
            print(f"Using fallback model with accuracy: {best_quantum_accuracy:.4f}")

# ===========================================
# STEP 5: Save Best Quantum Model
# ===========================================

quantum_file = None
if best_quantum_model is not None:
    try:
        quantum_package = {
            'model': best_quantum_model,
            'accuracy': best_quantum_accuracy,
            'parameters': best_quantum_params,
            'model_type': 'quantum',
            'model_name': 'best_vqc',
            'timestamp': timestamp,
            'training_samples': len(X_train_scaled),
            'test_samples': len(X_test_scaled),
            'features': X_train_scaled.shape[1],
            'qubits': 4
        }
        
        filename = f"best_quantum_model_{timestamp}.pkl"
        filepath = os.path.join(models_path, filename)
        
        with open(filepath, 'wb') as f:
            pickle.dump(quantum_package, f)
        
        quantum_file = filepath
        print(f"Saved: Best Quantum Model -> {filename} (Accuracy: {best_quantum_accuracy:.4f})")
        
    except Exception as e:
        print(f"Failed to save quantum model: {str(e)}")
else:
    print("No quantum model available to save")

# ===========================================
# STEP 6: Save Preprocessing Components
# ===========================================

print("\nSaving preprocessing components...")

try:
    preprocessing_package = {
        'scaler': scaler,  # Your StandardScaler
        'feature_names': [f'feature_{i}' for i in range(X_train_scaled.shape[1])],
        'dataset_info': {
            'name': dataset_name if 'dataset_name' in globals() else 'synthetic',
            'n_samples': len(X),
            'n_features': X.shape[1],
            'train_size': len(X_train_scaled),
            'test_size': len(X_test_scaled)
        },
        'timestamp': timestamp
    }
    
    filename = f"preprocessing_{timestamp}.pkl"
    filepath = os.path.join(models_path, filename)
    
    with open(filepath, 'wb') as f:
        pickle.dump(preprocessing_package, f)
    
    print(f"Saved: Preprocessing components -> {filename}")
    
except Exception as e:
    print(f"Failed to save preprocessing: {str(e)}")

# ===========================================
# STEP 7: Create Summary Report
# ===========================================

print("\nCreating summary report...")

summary = {
    'experiment_name': 'Healora Quantum ML Comparison',
    'timestamp': timestamp,
    'extraction_location': models_path,
    'classical_models': {
        name: {
            'accuracy': data['accuracy'],
            'file': f"{name}_{timestamp}.joblib"
        } for name, data in classical_models.items()
    },
    'quantum_model': {
        'accuracy': best_quantum_accuracy,
        'parameters': best_quantum_params,
        'file': f"best_quantum_model_{timestamp}.pkl" if quantum_file else None
    },
    'dataset_info': {
        'total_samples': len(X) if 'X' in globals() else 'unknown',
        'features': X.shape[1] if 'X' in globals() else 'unknown',
        'train_samples': len(X_train_scaled),
        'test_samples': len(X_test_scaled)
    },
    'best_classical': max(classical_models.items(), key=lambda x: x[1]['accuracy'])[0],
    'best_overall': 'classical' if max(classical_models.values(), key=lambda x: x['accuracy'])['accuracy'] > best_quantum_accuracy else 'quantum'
}

summary_filename = f"extraction_summary_{timestamp}.json"
summary_filepath = os.path.join(models_path, summary_filename)

with open(summary_filepath, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

# ===========================================
# STEP 8: Final Report
# ===========================================

print("\n" + "="*60)
print("MODEL EXTRACTION COMPLETE!!!!!")
print("="*60)
print(f"Location: {models_path}")
print(f"Summary: {summary_filename}")
print("\nExtracted Models:")

# List all files created
all_files = classical_files + ([quantum_file] if quantum_file else []) + [summary_filepath]
for i, filepath in enumerate(all_files, 1):
    filename = os.path.basename(filepath)
    file_size = os.path.getsize(filepath) / 1024  # KB
    print(f"   {i}. {filename} ({file_size:.1f} KB)")

print(f"\nBest Classical: {summary['best_classical']} ({classical_models[summary['best_classical']]['accuracy']:.4f})")
print(f"Best Quantum: {best_quantum_accuracy:.4f}")
print(f"Overall Best: {summary['best_overall'].title()}")

print(f"\nTo use these models later, navigate to: {models_path}")
print("Load with: joblib.load('filename.joblib') or pickle.load(open('filename.pkl', 'rb'))")

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Saved: logistic_regression -> logistic_regression_20250820_205939.joblib (Accuracy: 0.7333)
Saved: svm -> svm_20250820_205939.joblib (Accuracy: 0.9000)
Saved: random_forest -> random_forest_20250820_205939.joblib (Accuracy: 0.8000)

 Extracting Best Quantum Model...
Best quantum model found: 0.8000 accuracy
Retraining best quantum model...
Verified quantum model accuracy: 0.6000
Failed to save quantum model: Can't get local object 'VQC._get_interpret.<locals>.parity'

Saving preprocessing components...
Saved: Preprocessing components -> preprocessing_20250820_205939.pkl

Creating summary report...

MODEL EXTRACTION COMPLETE!!!!!
Location: C:\Users\safur\Projects\quantum-notebook\extracted_models
Summary: extraction_summary_20250820_205939.json

Extracted Models:
   1. logistic_regression_20250820_205939.joblib (1.0 KB)
   2. svm_20250820_205939.joblib (4.7 KB)
   3. random_forest_20250820_205939.joblib (246.4 KB)
   4. extraction_summary_20250820_205939.json (0.9 KB)

Best Classical: s